# Day 8: Haunted Wasteland

You're still riding a camel across Desert Island when you spot a sandstorm quickly approaching. When you turn to warn the Elf, she disappears before your eyes! To be fair, she had just finished warning you about _ghosts_ a few minutes ago.

One of the camel's pouches is labeled "maps" - sure enough, it's full of documents (your puzzle input) about how to navigate the desert. At least, you're pretty sure that's what they are; one of the documents contains a list of left/right instructions, and the rest of the documents seem to describe some kind of _network_ of labeled nodes.

It seems like you're meant to use the _left/right_ instructions to _navigate the network_. Perhaps if you have the camel follow the same instructions, you can escape the haunted wasteland!

After examining the maps for a bit, two nodes stick out: `AAA` and `ZZZ`. You feel like `AAA` is where you are now, and you have to follow the left/right instructions until you reach `ZZZ`.

This format defines each _node_ of the network individually. For example:

```
RL

AAA = (BBB, CCC)
BBB = (DDD, EEE)
CCC = (ZZZ, GGG)
DDD = (DDD, DDD)
EEE = (EEE, EEE)
GGG = (GGG, GGG)
ZZZ = (ZZZ, ZZZ)
```

Starting with `AAA`, you need to _look up the next element_ based on the next left/right instruction in your input. In this example, start with `AAA` and go _right_ (`R`) by choosing the right element of `AAA`, `_CCC_`. Then, `L` means to choose the _left_ element of `CCC`, `_ZZZ_`. By following the left/right instructions, you reach `ZZZ` in `_2_` steps.

Of course, you might not find `ZZZ` right away. If you run out of left/right instructions, repeat the whole sequence of instructions as necessary: `RL` really means `RLRLRLRLRLRLRLRL...` and so on. For example, here is a situation that takes `_6_` steps to reach `ZZZ`:

```
LLR

AAA = (BBB, BBB)
BBB = (AAA, ZZZ)
ZZZ = (ZZZ, ZZZ)
```

Starting at `AAA`, follow the left/right instructions. _How many steps are required to reach `ZZZ`?_

In [7]:
$sample_data = @'
LLR

AAA = (BBB, BBB)
BBB = (AAA, ZZZ)
ZZZ = (ZZZ, ZZZ)
'@ -split "`n" #NB: Polyglot seems to be OK with this, but in plain PoSH, it caused me pain as it had CRLF ("`r`n") line endings. FYI.

$input_data = Get-Content .\data.txt

In [11]:
function dostuff {
    [CmdletBinding()] Param(
        [Parameter(Mandatory=$false)]
        [switch] $sample,
        [Parameter(Mandatory=$false)]
        [switch] $part2
    )

    if($sample) {
        if(!$part2) {
            $data = $sample_data 
        } else {
            $data = @'
LR

11A = (11B, XXX)
11B = (XXX, 11Z)
11Z = (11B, XXX)
22A = (22B, XXX)
22B = (22C, 22C)
22C = (22Z, 22Z)
22Z = (22B, 22B)
XXX = (XXX, XXX)
'@ -split "`n"
        }
    } else {
        $data = $input_data
    }

    $directions = $data[0].trim()
    $direction_step = 0
    $pedometer = 0
    if(!$part2) {
        $positions = @('AAA')
    } else {
        $positions = [System.Collections.ArrayList]::new()
    }
    
    # setup the location map (& for part2 find the start positions)
    $map = @{}
    for($x = 2; $x -lt $data.count; $x++) {
        $temp = $data[$x].trim().split(' = (') # adding .trim() as some weird NPCs are sneaking in
        $temp2 = $temp[1].split(', ')
        $map.Add($temp[0], @{ 'L' = $temp2[0]; 'R' = $temp2[1].trim(')') })
        Write-Verbose "From data line $x, adding pos $($temp[0]) with L=$($temp2[0]) and R=$( $map[$temp[0]]['R'] )"
        if($part2 -and $temp[0][2] -eq 'A') {
            $positions.add($temp[0]) | Out-Null
        }
    }

    Write-Debug "Parsed $($map.keys.count) nodes and $($directions.length) directions [$directions]."
    if($part2) { Write-Debug "Also found $($positions.count) start positions ($($positions -join ', '))" }

    # follow the map!
    $continue_looking = $true
    while($continue_looking) {
        $part2_end = $true #gets falsed as soon as at least one position != xxZ
        for($x=0; $x -lt $positions.Count; $x++) {
            #update our location to our new location
            $newpos = $map[$positions[$x]]["$($directions[$direction_step])"]
            Write-Debug "Step $pedometer position #$x presently at $($positions[$x]) ($x of $($positions.Count-1)), will be heading $($directions[$direction_step]) to $newpos"
            $positions[$x] = $newpos
            # Solution checking
            if(!$part2) {
                if($positions[0] -eq 'ZZZ') {
                    $continue_looking = $false
                }
            } else {
                if($part2_end -and $positions[$x][2] -ne 'Z') { # the "$part2_end -and" should allow PoSH to skip the string slice/comparison, and improve performance.
                    $part2_end = $false
                }
            }
        }
        if($part2_end) {
            $continue_looking = $false
        }
        $pedometer++
        $direction_step++
        if($direction_step -ge $directions.length) {
            $direction_step = 0 #loop around per instructions
        }
        if(!$sample -and $pedometer -gt 10000 -or $sample -and $pedometer -gt 30) {
            Write-Error "Bailing out since we've looped too long..."
            break
        }

    }
    return $pedometer
}

In [12]:
Write-Output "Took $(dostuff -sample -Debug) steps to get to ZZZ."

DEBUG: Parsed 3 nodes and 3 directions [LLR].
DEBUG: Step 0 position #0 presently at AAA (0 of 0), will be heading L to BBB
Took 1 steps to get to ZZZ.


In [13]:
Write-Output "Took $(dostuff) steps to get to ZZZ."

Took 1 steps to get to ZZZ.


# Part Two

The sandstorm is upon you and you aren't any closer to escaping the wasteland. You had the camel follow the instructions, but you've barely left your starting position. It's going to take _significantly more steps_ to escape!

What if the map isn't for people - what if the map is for _ghosts_? Are ghosts even bound by the laws of spacetime? Only one way to find out.

After examining the maps a bit longer, your attention is drawn to a curious fact: the number of nodes with names ending in `A` is equal to the number ending in `Z`! If you were a ghost, you'd probably just _start at every node that ends with `A`_ and follow all of the paths at the same time until they all simultaneously end up at nodes that end with `Z`.

For example:

```
LR

11A = (11B, XXX)
11B = (XXX, 11Z)
11Z = (11B, XXX)
22A = (22B, XXX)
22B = (22C, 22C)
22C = (22Z, 22Z)
22Z = (22B, 22B)
XXX = (XXX, XXX)
```

Here, there are two starting nodes, `11A` and `22A` (because they both end with `A`). As you follow each left/right instruction, use that instruction to _simultaneously_ navigate away from both nodes you're currently on. Repeat this process until _all_ of the nodes you're currently on end with `Z`. (If only some of the nodes you're on end with `Z`, they act like any other node and you continue as normal.) In this example, you would proceed as follows:

- Step 0: You are at `11A` and `22A`.
- Step 1: You choose all of the _left_ paths, leading you to `11B` and `22B`.
- Step 2: You choose all of the _right_ paths, leading you to `_11Z_` and `22C`.
- Step 3: You choose all of the _left_ paths, leading you to `11B` and `_22Z_`.
- Step 4: You choose all of the _right_ paths, leading you to `_11Z_` and `22B`.
- Step 5: You choose all of the _left_ paths, leading you to `11B` and `22C`.
- Step 6: You choose all of the _right_ paths, leading you to `_11Z_` and `_22Z_`.

So, in this example, you end up entirely on nodes that end in `Z` after `_6_` steps.

Simultaneously start on every node that ends with `A`. _How many steps does it take before you're only on nodes that end with `Z`?_

In [14]:
Write-Output "Took $(dostuff -part2 -sample -Debug) steps to get to xxZ."

DEBUG: Parsed 8 nodes and 2 directions [LR].
DEBUG: Also found 2 start positions (11A, 22A)
DEBUG: Step 0 position #0 presently at 11A (0 of 1), will be heading L to 11B
DEBUG: Step 0 position #1 presently at 22A (1 of 1), will be heading L to 22B
DEBUG: Step 1 position #0 presently at 11B (0 of 1), will be heading R to 11Z
DEBUG: Step 1 position #1 presently at 22B (1 of 1), will be heading R to 22C
DEBUG: Step 2 position #0 presently at 11Z (0 of 1), will be heading L to 11B
DEBUG: Step 2 position #1 presently at 22C (1 of 1), will be heading L to 22Z
DEBUG: Step 3 position #0 presently at 11B (0 of 1), will be heading R to 11Z
DEBUG: Step 3 position #1 presently at 22Z (1 of 1), will be heading R to 22B
DEBUG: Step 4 position #0 presently at 11Z (0 of 1), will be heading L to 11B
DEBUG: Step 4 position #1 presently at 22B (1 of 1), will be heading L to 22C
DEBUG: Step 5 position #0 presently at 11B (0 of 1), will be heading R to 11Z
DEBUG: Step 5 position #1 presently at 22C (1 of 1

In [15]:
Write-Output "Took $(dostuff -part2) steps to get to xxZ."

dostuff: Bailing out since we've looped too long...
Took 10001 steps to get to xxZ.


Error: Command failed: SubmitCode: Write-Output "Took $(dostuff -part2) steps to get  ...

Welp, this one kicked my but. Tried converting it to vanilla PoSH for more performance, turns out it was actually _slower_, haha. Anyway, might come back to this one.